# Data Loading

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed May 11 14:23:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from keras.layers import Dense
from keras.models import Sequential
from keras.initializers import he_normal, normal

# from deepreplay.datasets.parabola import load_data
# from deepreplay.callbacks import ReplayData
# from deepreplay.replay import Replay
# from deepreplay.plot import compose_animations, compose_plots
import random
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


In [5]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "/content/drive/MyDrive/9.822"

Mounted at /content/drive


In [6]:
%cd "/content/drive/MyDrive/9.822"

/content/drive/MyDrive/9.822


In [39]:
# Loads the CSV data
# spam
df_spam = pd.read_csv('spambase.data', header=None)

# breast cancer
df_breast = pd.read_csv('breast_cancer.csv')
df_breast.drop(df_breast.columns[df_breast.columns.str.contains('unnamed', case = False)], axis = 1, inplace = True)

# german 
df_german = pd.read_csv('german.csv')

# austrian
df_australian = pd.read_csv('australian.csv')

# hill-valley
df_hill_valley = pd.read_csv('hill-valley.csv')


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,Class
0,39.02,36.49,38.20,38.85,39.38,39.74,37.02,39.53,38.81,38.79,...,36.62,36.92,38.80,38.52,38.07,36.73,39.46,37.50,39.10,0
1,1.83,1.71,1.77,1.77,1.68,1.78,1.80,1.70,1.75,1.78,...,1.80,1.79,1.77,1.74,1.74,1.80,1.78,1.75,1.69,1
2,68177.69,66138.42,72981.88,74304.33,67549.66,69367.34,69169.41,73268.61,74465.84,72503.37,...,73438.88,71053.35,71112.62,74916.48,72571.58,66348.97,71063.72,67404.27,74920.24,1
3,44889.06,39191.86,40728.46,38576.36,45876.06,47034.00,46611.43,37668.32,40980.89,38466.15,...,42625.67,40684.20,46960.73,44546.80,45410.53,47139.44,43095.68,40888.34,39615.19,0
4,5.70,5.40,5.28,5.38,5.27,5.61,6.00,5.38,5.34,5.87,...,5.17,5.67,5.60,5.94,5.73,5.22,5.30,5.73,5.91,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,13.00,12.87,13.27,13.04,13.19,12.53,14.31,13.33,13.63,14.55,...,12.48,12.15,13.15,12.35,13.58,13.86,12.88,13.87,13.51,1
1208,48.66,50.11,48.55,50.43,50.09,49.67,48.95,48.65,48.63,48.61,...,46.93,49.61,47.16,48.17,47.94,49.81,49.89,47.43,47.77,0
1209,10160.65,9048.63,8994.94,9514.39,9814.74,10195.24,10031.47,10202.28,9152.99,9591.75,...,9068.11,9191.80,9275.04,9848.18,9074.17,9601.74,10366.24,8997.60,9305.77,1
1210,34.81,35.07,34.98,32.37,34.16,34.03,33.31,32.48,35.63,32.48,...,32.76,35.03,32.89,31.91,33.85,35.28,32.49,32.83,34.82,1


# Classification 

**Classifiers**

In [9]:
def trainer(classifier, x_train, y_train, x_test, y_test):
  classifier.fit(x_train, y_train)
  y_pred = classifier.predict(x_test)
  return accuracy_score(y_test, y_pred)

#Using Logistic Regression Algorithm to the Training Set
classifier_lr = LogisticRegression(random_state = 0)

#Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

#Using SVC method of svm class to use Support Vector Machine Algorithm
classifier_svc_l = SVC(kernel = 'linear', random_state = 0)

#Using SVC method of svm class to use Kernel SVM Algorithm
classifier_svc_rbf = SVC(kernel = 'rbf', random_state = 0)

#Using GaussianNB method of naïve_bayes class to use Naïve Bayes Algorithm
classifier_gnb = GaussianNB()

#Using DecisionTreeClassifier of tree class to use Decision Tree Algorithm
classifier_cart = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

#Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)

# using Perceptron method 
classifier_perceptron = Perceptron(tol=1e-3, random_state=0)

# using MLP classifier
classifier_mlp = MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2)

**Neural Net**

In [10]:
def nn_trainer(input_dim, X_train, y_train, X_test, y_test):
    
  he_initializer = he_normal(seed=42)
  normal_initializer = normal(seed=42)

  model = Sequential()
  # Hidden layer with 10 units, taking the 57 features as inputs
  model.add(Dense(input_dim=input_dim,
                  units=10,
                  kernel_initializer=he_initializer,
                  activation='tanh'))

  # Added layer to allow plotting the feature space
  # It has 2 units and uses a LINEAR activation, so the network will also learn the
  # mapping from 10-dimensions to 2-dimensions
  model.add(Dense(units=2,
                  kernel_initializer=normal_initializer,
                  activation='linear',
                  name='hidden'))

  # Typical output layer for binary classification
  model.add(Dense(units=1,
                  kernel_initializer=normal_initializer,
                  activation='sigmoid',
                  name='output'))

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc'])

  print(model.summary())

  # model.fit(X, y, epochs=100, batch_size=16, callbacks=[replaydata])
  # model.fit(X_train, y_train, epochs=20, batch_size=16)
  # results = model.evaluate(X_test, y_test)
  # print("test loss, test acc:", results)

**SpamBase**

In [1]:
group_name = 'spam'
# The first 57 columns are features
# The last column has the correct labels (targets)
X, Y = df_spam.iloc[:, :57].values, df_spam.iloc[:, 57].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,
                                                    random_state = 0)

# Scale the features, as the original values have wide ranges
X = StandardScaler().fit_transform(X)
#replaydata = ReplayData(X, y, filename='spambase_dataset.h5', group_name=group_name)
# nn_trainer(57, X_train, y_train, X_test, y_test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# evaluation 
print(trainer(classifier_rf, X_train, Y_train, X_test, Y_test))
print(cross_val_score(estimator = classifier_rf, X = X_train, y = Y_train, cv = 10).mean())

mapper_spam = {'Spam': [X, Y]}
df_spam

NameError: ignored

**Breast Cancer Wisconsin (Diagnostic)**

In [12]:
X = df_breast.iloc[:, 2:32].values
Y = df_breast.iloc[:, 1].values

#Encoding categorical data values
labelencoder_Y = LabelEncoder()
Y = labelencoder_Y.fit_transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

#Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# evaluation
print(trainer(classifier_svc_l, X_train, Y_train, X_test, Y_test))
print(cross_val_score(estimator = classifier_svc_l, X = X_train, y = Y_train, cv = 10).mean())
mapper_breast = {'Breast': [X, Y]}

0.972027972027972
0.9743632336655592


**German Credit Dataset**

In [13]:
X = df_german.iloc[:, 1:]
Y = df_german.iloc[:, 0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 23, stratify=Y)
# normalzie
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# evaluation
# print(trainer(Perceptron(tol=1e-3, random_state=0), X_train, Y_train, X_test, Y_test))
print(cross_val_score(estimator = classifier_svc_l, X = X_train, y = Y_train, cv = 10).mean())

mapper_german = {'German': [X, Y]}


0.76


**Australian Credit Approval Dataset**

In [14]:
X = df_australian.iloc[:, 0:14]
Y = df_australian.iloc[:, 14]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 23, stratify=Y)
# normalize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# evaluation
print(trainer(classifier_svc_l, X_train, Y_train, X_test, Y_test))
print(cross_val_score(estimator = classifier_svc_l, X = X_train, y = Y_train, cv = 10).mean())

mapper_australian = {'Australian': [X, Y]}


0.8840579710144928
0.8459090909090909


**Hill-Valley Dataset**

In [15]:
X = df_hill_valley.iloc[:, 0:100]
Y = df_hill_valley.iloc[:, 100]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 23, stratify=Y)
# normalize
X_train = normalize(X_train)
X_test = normalize(X_test)

# evaluation
print(trainer(classifier_svc_rbf, X_train, Y_train, X_test, Y_test))
print(cross_val_score(estimator = classifier_svc_rbf, X = X_train, y = Y_train, cv = 10).mean())

mapper_hill_valley = {'Hill-Valley': [X, Y]}



0.9876543209876543
0.9907216494845361


In [16]:
mapper = {}
mapper.update(mapper_spam)
mapper.update(mapper_breast)
mapper.update(mapper_german)
mapper.update(mapper_australian)
mapper.update(mapper_hill_valley)

# Heuristic Machine Truth Serum (HMTS)

In [21]:
def train_classifiers(mapper):
  for k, vals in mapper.items():
    vals[0].fit(vals[1], vals[3])
    print('Finished trainining ' + k)
  return mapper

def peer_prediction(classifier, classifier_result, x, mapper):
  peer_prediction_estimation = 0 
  for k, vals in mapper.items():
    if k == classifier:
      continue
    result = vals[0].predict(x)
    if classifier_result == result:
      peer_prediction_estimation += 1
  
  return peer_prediction_estimation / len(mapper.keys())

def calc_posterior_and_majority(mapper, answer, x, Large):
  posterior = 0
  counter_0 = 0
  counter_1 = 0
  majority = None
  high_disagreement = False
  for k, vals in mapper.items():
    pred = vals[0].predict(x)
    if pred == answer:
      posterior += 1
    if pred == 0:
      counter_0 += 1
    else:
      counter_1 += 1

  if counter_0 > counter_1:
    majority = 0
  else:
    majority = 1

  posterior = posterior / len(mapper.keys())
  if Large:
    if counter_0 == 8 or counter_0 == 7:
      high_disagreement = True
  else:
    if counter_0 == 2 or counter_0 == 3:
      high_disagreement = True
  return posterior, majority, high_disagreement
    
def HMTS(mapper, one_regressor=False, Large=True):
  # train each classifier
  mapper = train_classifiers(mapper)

  # for each classifier
  new_mapper = mapper.copy()
  for k, vals in mapper.items():
    # for each data point in X_train
    print('Working on classifier ' + k)
    peer_prediction_0_x = []
    peer_prediction_0_y = []
    peer_prediction_1_x = []
    peer_prediction_1_y = []
    for i in vals[1]:
      # compute peer prediction information
      x = i.reshape(1, -1)
      pred = vals[0].predict(x)
      peer_pred = peer_prediction(k, pred, x, mapper)
      if one_regressor:
        peer_prediction_0_x.append(i)
        peer_prediction_0_y.append(peer_pred)
      else:
        if pred == 0:
          peer_prediction_0_x.append(i)
          peer_prediction_0_y.append(peer_pred)
        elif pred == 1:
          peer_prediction_1_x.append(i)
          peer_prediction_1_y.append(peer_pred)

    assert len(peer_prediction_0_x) == len(peer_prediction_0_y), "lengths must be equal"
    if not one_regressor:
      assert len(peer_prediction_1_x) == len(peer_prediction_1_y), "lengths must be equal"
      assert len(peer_prediction_0_x) + len(peer_prediction_1_x) == len(vals[1]), "lengths must sum to len(X_train)"

    if one_regressor:
      belief_regressor_0 = RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', random_state = 0).fit(
          peer_prediction_0_x, peer_prediction_0_y)
      new_mapper[k].extend([belief_regressor_0, peer_prediction_0_x, peer_prediction_0_y])

    else:
      if len(peer_prediction_0_x) != 0:
        belief_regressor_0 = RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', random_state = 0).fit(peer_prediction_0_x, peer_prediction_0_y)
      else:
        belief_regressor_0 = RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', random_state = 0)
      if len(peer_prediction_1_x) != 0:
        belief_regressor_1 = RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', random_state = 0).fit(peer_prediction_1_x, peer_prediction_1_y)
      else:
        belief_regressor_1 = RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', random_state = 0)

      new_mapper[k].extend([belief_regressor_0, peer_prediction_0_x, peer_prediction_0_y, 
                            belief_regressor_1, peer_prediction_1_x, peer_prediction_1_y])
    
  # evaluation
  test_data = new_mapper['RandomForest_100'][2]
  right_answers = new_mapper['RandomForest_100'][4].reset_index()
  hmts_answers = []
  majority_answers = []
  ground_truth = []
  for inx, x in enumerate(test_data):
    x_transformed = x.reshape(1, -1)
    prior_0 = 0
    posterior_0 = 0
    for k, vals in new_mapper.items():
      pred = vals[0].predict(x_transformed)
      if one_regressor:
        peer_pred = vals[6].predict(x_transformed)
      else:
        if pred == 0:
          peer_pred = vals[6].predict(x_transformed)
        elif pred == 1:
          peer_pred = 1 - vals[9].predict(x_transformed)
      prior_0 += peer_pred

    prior_0 = prior_0 / len(new_mapper.keys())
    posterior_0, majority, high_disagreement = calc_posterior_and_majority(new_mapper, 0, x_transformed, Large)
    if high_disagreement:
      if posterior_0 > prior_0:
        hmts_answers.append(0)
      else:
        hmts_answers.append(1)
      majority_answers.append(majority)
      ground_truth.append(right_answers.iloc[inx, 1])
    else:
      hmts_answers.append(majority)
      majority_answers.append(majority)
      ground_truth.append(right_answers.iloc[inx, 1])
  print('Done')
  return hmts_answers, majority_answers, ground_truth

# Discriminative Machine Truth Serum (DMTS)

In [22]:
def calc_majority(mapper, x, Large=True):
  counter_0 = 0
  counter_1 = 0
  majority_0 = None
  high_disagreement = False
  for k, vals in mapper.items():
    pred = vals[0].predict(x)
    if pred == 0:
      counter_0 += 1
    elif pred == 1:
      counter_1 += 1
  if counter_0 > counter_1:
    majority_0 = 0
  else:
    majority_0 = 1
  
  if Large:
    if counter_0 == 8 or counter_0 == 7:
      high_disagreement = True
  else:
    if counter_0 == 2 or counter_0 == 3:
      high_disagreement = True
  return majority_0, high_disagreement
  

def DMTS(mapper, Large=True):
  # train each classifier
  mapper = train_classifiers(mapper)
  train_data = mapper['RandomForest_100'][1]
  y_train = mapper['RandomForest_100'][3].reset_index()
  majority_right = []
  for inx, x in enumerate(train_data):
    x_transformed = x.reshape(1, -1)
    majority, high_disagreement = calc_majority(mapper, x_transformed)
    right_answer = y_train.iloc[inx, 1]
    if majority == right_answer:
      majority_right.append(0)
    else:
      majority_right.append(1)

  # fit majority checker classifier
  classifier_rf = RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0)
  classifier_rf.fit(train_data, majority_right)

  test_data = mapper['RandomForest_100'][2]
  y_test = mapper['RandomForest_100'][4].reset_index()
  ground_truth = []
  majority_answers = []
  dmts_answers = [] 
  for inx, x in enumerate(test_data):
    x_transformed = x.reshape(1, -1)
    pred = classifier_rf.predict(x_transformed)
    majority, high_disagreement = calc_majority(mapper, x_transformed, Large)
    if high_disagreement:
      # print('high disagreement encountered')
      majority_answers.append(majority)
      ground_truth.append(y_test.iloc[inx, 1])
      if pred == 0:
        dmts_answers.append(majority)
      else:
        if majority == 0:
          dmts_answers.append(1)
        else:
          dmts_answers.append(0)
    else:
      dmts_answers.append(majority)
      majority_answers.append(majority)
      ground_truth.append(y_test.iloc[inx, 1])
  return dmts_answers, majority_answers, ground_truth

# Initialize Datasets

In [23]:
def data_initializer(X_train, X_test, Y_train, Y_test, Y, Large=True):
  data_mapper = None
  if Large:
    data_mapper = {'Decision_Tree': [DecisionTreeClassifier(criterion = 'entropy', random_state = 0), X_train, X_test, Y_train, Y_test, Y], 
                      'Naive_Bayes': [GaussianNB(), X_train, X_test, Y_train, Y_test, Y],
                      'Perceptron': [Perceptron(tol=1e-5, random_state=0), X_train, X_test, Y_train, Y_test, Y], 
                      'Logistic_500': [LogisticRegression(random_state = 0, max_iter=500), X_train, X_test, Y_train, Y_test, Y], 
                      'Logistic_400': [LogisticRegression(random_state = 0, max_iter=400), X_train, X_test, Y_train, Y_test, Y],
                      'Logistic_300': [LogisticRegression(random_state = 0, max_iter=300), X_train, X_test, Y_train, Y_test, Y], 
                      'RandomForest_15': [RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0), 
                                          X_train, X_test, Y_train, Y_test, Y], 
                      'RandomForest_50': [RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 0), 
                                          X_train, X_test, Y_train, Y_test, Y], 
                      'RandomForest_100': [RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0), 
                                          X_train, X_test, Y_train, Y_test, Y], 
                      'SVM_rbf': [SVC(kernel = 'rbf', random_state = 0), X_train, X_test, Y_train, Y_test, Y], 
                      'SVM_linear': [SVC(kernel = 'linear', random_state = 0), X_train, X_test, Y_train, Y_test, Y], 
                      'SVM_poly': [SVC(kernel = 'poly', random_state = 0), X_train, X_test, Y_train, Y_test, Y],
                      'MLP_300': [MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2), X_train, X_test, Y_train, Y_test, Y],
                      'MLP_400': [MLPClassifier(random_state=0, max_iter=400, verbose=False, tol=1e-2), X_train, X_test, Y_train, Y_test, Y],
                      'MLP_500': [MLPClassifier(random_state=0, max_iter=500, verbose=False, tol=1e-2), X_train, X_test, Y_train, Y_test, Y]}
  else:
    data_mapper = {'Perceptron_1': [Perceptron(tol=1e-3, random_state=0), X_train, X_test, Y_train, Y_test, Y], 
                    'RandomForest_100': [RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 0), 
                                        X_train, X_test, Y_train, Y_test, Y], 
                    'SVM_1': [SVC(kernel = 'rbf', random_state = 0), X_train, X_test, Y_train, Y_test, Y], 
                    'Logistic_1': [LogisticRegression(random_state = 0, max_iter=300), X_train, X_test, Y_train, Y_test, Y], 
                    'MLP_300': [MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2), X_train, X_test, Y_train, Y_test, Y]}
  return data_mapper


def train_mts(mapper):
  results = {}
  for mapper_name in mapper.keys():
    print("-------------------------------------------------------------------")
    print('Working on dataset', mapper_name)
    Y_06 = []
    Y_08 = []
    Y_1 = []
    for inx, val in enumerate(mapper[mapper_name][1]):
      if val == 0:
        new_val_06 = random.choices([0, 1], weights = [0.94, 0.06])
        new_val_08 = random.choices([0, 1], weights = [0.92, 0.08])
        new_val_1 = random.choices([0, 1], weights = [0.9, 0.1])
      elif val == 1:
        new_val_06 = random.choices([1, 0], weights = [0.94, 0.06])
        new_val_08 = random.choices([1, 0], weights = [0.92, 0.08])
        new_val_1 = random.choices([1, 0], weights = [0.9, 0.1])

      Y_06.append(new_val_06[0])
      Y_08.append(new_val_08[0])
      Y_1.append(new_val_1[0])

    Y_06 = pd.Series(Y_06)
    Y_08 = pd.Series(Y_08)
    Y_1 = pd.Series(Y_1)

    X = mapper[mapper_name][0]
    Y = pd.Series(mapper[mapper_name][1])

    X_train, X_test, Y_train_06, Y_test_06 = train_test_split(X, Y_06, test_size = 0.2, random_state = 23, stratify=Y_06)
    X_train, X_test, Y_train_08, Y_test_08 = train_test_split(X, Y_08, test_size = 0.2, random_state = 23, stratify=Y_08)
    X_train, X_test, Y_train_1, Y_test_1 = train_test_split(X, Y_1, test_size = 0.2, random_state = 23, stratify=Y_1)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 23, stratify=Y)
    # normalize
    if mapper_name == 'Hill-Valley':
      X_train = normalize(X_train)
      X_test = normalize(X_test)
    else:
      sc = StandardScaler()
      X_train = sc.fit_transform(X_train)
      X_test = sc.transform(X_test)

    # data_mapper = {'Perceptron_06': [Perceptron(tol=1e-3, random_state=0), X_train, X_test, Y_train_06, Y_test_06, Y], 
    #                     'Perceptron_08': [Perceptron(tol=1e-3, random_state=0), X_train, X_test, Y_train_08, Y_test_08, Y],
    #                     'Perceptron_1': [Perceptron(tol=1e-3, random_state=0), X_train, X_test, Y_train_1, Y_test_1, Y], 
    #                     'Logistic_06': [LogisticRegression(random_state = 0, max_iter=500), X_train, X_test, Y_train_06, Y_test_06, Y], 
    #                     'Logistic_08': [LogisticRegression(random_state = 0, max_iter=500), X_train, X_test, Y_train_08, Y_test_08, Y],
    #                     'Logistic_1': [LogisticRegression(random_state = 0, max_iter=500), X_train, X_test, Y_train_1, Y_test_1, Y], 
    #                     'RandomForest_06': [RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0), 
    #                                         X_train, X_test, Y_train_06, Y_test_06, Y], 
    #                     'RandomForest_08': [RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0), 
    #                                         X_train, X_test, Y_train_08, Y_test_08, Y], 
    #                     'RandomForest_1': [RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0), 
    #                                         X_train, X_test, Y_train_1, Y_test_1, Y], 
    #                     'SVM_06': [SVC(kernel = 'rbf', random_state = 0), X_train, X_test, Y_train_06, Y_test_06, Y], 
    #                     'SVM_08': [SVC(kernel = 'rbf', random_state = 0), X_train, X_test, Y_train_08, Y_test_08, Y], 
    #                     'SVM_1': [SVC(kernel = 'rbf', random_state = 0), X_train, X_test, Y_train_1, Y_test_1, Y],
    #                     'MLP_06': [MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2), X_train, X_test, Y_train_06, Y_test_06, Y],
    #                     'MLP_08': [MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2), X_train, X_test, Y_train_08, Y_test_08, Y],
    #                     'MLP_1': [MLPClassifier(random_state=0, max_iter=300, verbose=False, tol=1e-2), X_train, X_test, Y_train_1, Y_test_1, Y]}


    data_mapper = data_initializer(X_train, X_test, Y_train, Y_test, Y, Large=True)
    hmts, majority, ground_truth = HMTS(data_mapper, one_regressor=False, Large=True)
    print('HMTS Score:', accuracy_score(ground_truth, hmts))
    print('Majority Score:', accuracy_score(ground_truth, majority))

    data_mapper = data_initializer(X_train, X_test, Y_train, Y_test, Y, Large=True)
    dmts, majority, ground_truth = DMTS(data_mapper, Large=True)
    print('DMTS Score:', accuracy_score(ground_truth, dmts))
    print('Majority Score:', accuracy_score(ground_truth, majority))
    print('Random Forest Score:', accuracy_score(Y_test, RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0).fit(
        X_train, Y_train).predict(X_test)))
    print('Adaboost Score:', accuracy_score(Y_test, AdaBoostClassifier(n_estimators=15).fit(
        X_train, Y_train).predict(X_test)))
    # results[mapper_name] = [hmts, dmts, majority, ground_truth]
  return results

In [24]:
results = train_mts(mapper)

-------------------------------------------------------------------
Working on dataset Spam
Finished trainining Decision_Tree
Finished trainining Naive_Bayes
Finished trainining Perceptron
Finished trainining Logistic_500
Finished trainining Logistic_400
Finished trainining Logistic_300
Finished trainining RandomForest_15
Finished trainining RandomForest_50
Finished trainining RandomForest_100
Finished trainining SVM_rbf
Finished trainining SVM_linear
Finished trainining SVM_poly
Finished trainining MLP_300
Finished trainining MLP_400
Finished trainining MLP_500
Working on classifier Decision_Tree
Working on classifier Naive_Bayes
Working on classifier Perceptron
Working on classifier Logistic_500
Working on classifier Logistic_400
Working on classifier Logistic_300
Working on classifier RandomForest_15
Working on classifier RandomForest_50
Working on classifier RandomForest_100
Working on classifier SVM_rbf
Working on classifier SVM_linear
Working on classifier SVM_poly
Working on cla

In [1]:
results

NameError: ignored